<a href="https://colab.research.google.com/github/dominiquebuford/DL-individual/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#we need to define the center of the body and then process the data points as their distance from the center

In [ ]:
import numpy as np

In [ ]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%

In [ ]:
from google.colab import files

# Upload the JSON key file
uploaded = files.upload()
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/service-account-key.json'

Saving service-account-key.json to service-account-key.json


In [ ]:
!gsutil cp gs://dl-individual-project/annotations/mpii_human_pose.csv /content/human_pose.csv

Copying gs://dl-individual-project/annotations/mpii_human_pose.csv...
/ [1 files][  3.7 MiB/  3.7 MiB]                                                
Operation completed over 1 objects/3.7 MiB.                                      


In [ ]:
#update csv file to only contain the images I will be using
from google.cloud import storage
import pandas as pd

bucket_name = 'dl-individual-project'
images_folder = 'images/'
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blobs = bucket.list_blobs(prefix = images_folder)
blob_names = [blob.name[len(images_folder):] for blob in blobs]

df= pd.read_csv("human_pose.csv")

df_filtered = df[df['NAME'].isin(blob_names)]

#calculate point from center


#df_filtered.to_csv("filtered_annotations.csv")

#blob = bucket.blob("annotations/filtered_annotations.csv")

#blob.upload_from_filename("filtered_annotations.csv")

In [ ]:
df_filtered.replace('football, competitive', 'football', inplace = True)
df_filtered.replace('football, touch, flag', 'football', inplace = True)
df_filtered.replace('basketball, game (Taylor Code 490)', 'basketball', inplace = True)
df_filtered.replace('boxing, sparring', 'boxing', inplace = True)
df_filtered.replace('boxing, in ring, general', 'boxing', inplace = True)

df_filtered['Activity'].value_counts()

<ipython-input-6-cb589e78a665>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.replace('football, competitive', 'football', inplace = True)
<ipython-input-6-cb589e78a665>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.replace('football, touch, flag', 'football', inplace = True)
<ipython-input-6-cb589e78a665>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.replace('basketball, game (Taylor Code 49

rowing, stationary                                       118
skateboarding                                            108
basketball                                                95
golf                                                      89
skiing, downhill                                          63
soccer                                                    59
softball, general                                         54
bowling                                                   38
boxing                                                    36
hacky sack                                                34
tennis, hitting balls, non-game play, moderate effort     30
frisbee                                                   28
football                                                  28
track and field                                           27
rollerblading                                             26
lacrosse                                                  13
Name: Activity, dtype: i

In [ ]:
#replace the -1 in each category with the average value for each class
#excluded_columns = ['center_X','center_Y','ID', 'NAME', 'Scale', 'Activity', 'Category']
#check_columns = [column for column in df_filtered.columns if column not in excluded_columns]

#for column in check_columns:
  #avg_per_activity = df_filtered[df_filtered[column] != -1].groupby('Activity')[column].mean()
  #for activity_label, avg_value in avg_per_activity.items():
            #mask = (df_filtered['Activity'] == activity_label) & (df_filtered[column] == -1)
            #df_filtered.loc[mask, column] = int(avg_value)



In [ ]:
df_filtered = df_filtered[~((df_filtered['l hip_X'] ==-1) | (df_filtered['r hip_X']==-1))]

In [ ]:
def calcLengthOfBody(row, excluded_columns):
  x_values = [val for columnName, val in row.items() if (columnName not in excluded_columns) and (columnName.find('_X')!=-1)]
  y_values = [val for columnName, val in row.items() if (columnName not in excluded_columns) and (columnName.find('_Y')!=-1)]
  point1 = np.array([max(x_values), max(y_values)])
  point2 = np.array([min(x_values), min(y_values)])
  euclidean_distance = np.linalg.norm(point1 - point2)
  return euclidean_distance

In [ ]:
def normalize(row):
  row_df = pd.DataFrame(row).transpose()
  excluded_columns = ['center_X','center_Y','ID', 'NAME', 'Scale', 'Activity', 'Category']

  lenBody = calcLengthOfBody(row, excluded_columns)
  #calculate distance from center
  for column in row_df.columns:
    if column not in excluded_columns:
      if column.find('_X') != -1:
        row_df[column] = (row_df[column] - row_df['center_X'])/lenBody
      if column.find('_Y') != -1:
        row_df[column] = (row_df[column] - row_df['center_Y'])/lenBody

  return row_df.squeeze()




In [ ]:
#check if both hip sides are visible

#df_filtered = df_filtered[~((df_filtered['r hip_X'] == -1) | (df_filtered['l hip_X'] == -1))]

df_filtered['center_X'] = (df_filtered['r hip_X'] + df_filtered['l hip_X'])/2
df_filtered['center_Y'] = (df_filtered['r hip_Y'] + df_filtered['l hip_Y']) / 2

df_normalized = df_filtered.apply(normalize, axis = 1)


In [ ]:
excluded_columns = ['center_X','center_Y','ID', 'NAME', 'Scale', 'Category']
df_final = df_normalized[df_normalized.columns.difference(excluded_columns)].copy()

In [ ]:
df_filtered.to_csv("filtered_annotations.csv")

blob = bucket.blob("annotations/filtered_annotations.csv")

blob.upload_from_filename("filtered_annotations.csv")

df_normalized.to_csv("normalized_annotations_with_names.csv")

blob = bucket.blob("annotations/normalized_annotations_with_names.csv")

blob.upload_from_filename("normalized_annotations_with_names.csv")

df_final.to_csv("normalized_annotations.csv")

blob = bucket.blob("annotations/normalized_annotations.csv")

blob.upload_from_filename("normalized_annotations.csv")

